<a href="https://colab.research.google.com/github/armandossrecife/teste/blob/main/git_teste_cassandra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importa pacotes que serão usados

In [10]:
import pandas as pd

In [11]:
#!rm -rf cassandra

In [12]:
def checa_indice_language(relacao_linguagens):
  indice=-1
  for i, each in enumerate(relacao_linguagens):
    if "Language" in each:
      indice = i
      break
  return indice

In [13]:
!sudo apt-get install cloc

Reading package lists... Done
Building dependency tree       
Reading state information... Done
cloc is already the newest version (1.82-1).
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.


# Clona repositorio principal

In [14]:
# clona o repositorio
!git clone https://github.com/apache/cassandra.git

Cloning into 'cassandra'...
remote: Enumerating objects: 381213, done.
remote: Counting objects: 100% (338/338), done.
remote: Compressing objects: 100% (211/211), done.
remote: Total 381213 (delta 87), reused 232 (delta 55), pack-reused 380875
Receiving objects: 100% (381213/381213), 381.87 MiB | 26.30 MiB/s, done.
Resolving deltas: 100% (221864/221864), done.


In [15]:
# Primeiro commit 
!cd cassandra && git rev-list --all | tail -n 1

1f91e99223b0d1b7ed8390400d4a06ac08e4aa85


In [16]:
# Ultimo commit 
!cd cassandra && git rev-list --all | head -n 1

6d62d2f4f2efb144824a433c0f8048c8a0eeed22


# Cria um repositorio apontando padra o 1o. commit

In [22]:
# Cria uma nova pasta para referenciar apenas os dados do 1o commit
!cp -R cassandra cassandra_c1

In [23]:
# git checkout $( git rev-list --all | tail -n 1 )
!cd cassandra_c1 && git checkout 1f91e99223b0d1b7ed8390400d4a06ac08e4aa85

Note: switching to '1f91e99223b0d1b7ed8390400d4a06ac08e4aa85'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 1f91e99223 git-svn-id: https://svn.apache.org/repos/asf/incubator/cassandra/trunk@749218 13f79535-47bb-0310-9956-ffa450edef68


In [24]:
!cd cassandra_c1 && git status

HEAD detached at 1f91e99223
nothing to commit, working tree clean


In [25]:
# Quantidade de arquivos na pasta cassandra
qtd_arquivos = !find cassandra_c1 -type f | wc -l

# Quantidade de commits do repositorio cassandra
qtd_commits = !cd cassandra_c1 && git log --oneline | grep -c .

# Tamanho (MB) da pasta cassandra
tamanho_pasta_mb = !du -hs --block-size=1M cassandra_c1 | cut -f1

# Quantidade de colaboradores (devs que commitaram pelo menos uma vez no projeto cassandra)
qtd_colaboradores_commits = !cd cassandra_c1 && git log --pretty=format:"%ae" | sort -u | grep -v "noreply.github.com" | wc -l

# Data do primeiro commit do projeto cassandra
data_primeiro_commit =!cd cassandra_c1 && git log --reverse --pretty=format:%ad --date=short | head -1

# Data do ultimo commit do projeto cassandra
data_ultimo_commit = !cd cassandra_c1 && git log -1 --pretty=format:%ad --date=short

dict_info_repo = {
  'nome':'cassandra_c1',
	'qtd_arquivos':qtd_arquivos,
	'qtd_commits':qtd_commits,
	'tamanho_pasta_mb':tamanho_pasta_mb,
	'qtd_colaboradores_commits':qtd_colaboradores_commits,
	'data_primeiro_commit':data_primeiro_commit,
	'data_ultimo_commit':data_ultimo_commit,
}

df_info_repo = pd.DataFrame(dict_info_repo)
df_info_repo

In [ ]:
# Detalha os tipos de arquivos que aparecem no repositório do cassandra
relacao_linguagens = !cd cassandra_c1 && cloc .
#relacao_linguagens[checa_indice_language(relacao_linguagens):]
linguagens_encontradas = relacao_linguagens[(checa_indice_language(relacao_linguagens)+2):]

lista_linguagens, lista_files, lista_blank, lista_comment, lista_code = list(), list(), list(), list(), list()

l_temp = []
vl_temp = []
for linha in linguagens_encontradas:
  elementos = linha.split(' ')
  novos_elementos = []
  linguagem_encontrada = []
  valores_da_linguagem = []
  for item in elementos:
    if item != '':
      novos_elementos.append(item)
  linguagem_encontrada.append(novos_elementos[:-4])
  valores_da_linguagem.append(novos_elementos[-4:])
  #print(linguagem_encontrada, valores_da_linguagem)
  l_temp = l_temp + linguagem_encontrada
  vl_temp = vl_temp + valores_da_linguagem

l_temp = l_temp[:-3]
for i, each in enumerate(l_temp):
  print(i,each)

vl_temp = vl_temp[:-3]
for i,each in enumerate(vl_temp):
  print(i,each)
  lista_files.append(each[0])
  lista_blank.append(each[1]) 
  lista_comment.append(each[2]) 
  lista_code.append(each[3])

In [33]:
dict_dados_linguagens = {
    'linguagens':l_temp, 
    'files':lista_files, 
    'blank':lista_blank,
    'comment':lista_comment,
    'code':lista_code
}

df_dados_linguagens = pd.DataFrame(dict_dados_linguagens)

df_dados_linguagens['files'] = df_dados_linguagens['files'].apply(lambda x: int(x))
df_dados_linguagens['blank'] = df_dados_linguagens['blank'].apply(lambda x: int(x))	
df_dados_linguagens['comment'] = df_dados_linguagens['comment'].apply(lambda x: int(x))	
df_dados_linguagens['code'] = df_dados_linguagens['code'].apply(lambda x: int(x))

df_dados_linguagens

,linguagens,files,blank,comment,code
0,[Java],313,8271,16341,49049
1,"[ANTLR, Grammar]",4,121,201,425


# Cria um repositório apontando para o último commit

In [37]:
# Cria uma nova pasta para referenciar apenas os dados do 1o commit
!cp -R cassandra cassandra_cf

In [ ]:
# git checkout $( git rev-list --all | head -n 1 )
!cd cassandra_cf && git checkout 6d62d2f4f2efb144824a433c0f8048c8a0eeed22

In [39]:
!cd cassandra_cf && git status

HEAD detached at 6d62d2f4f2
nothing to commit, working tree clean


In [40]:
# Quantidade de arquivos na pasta cassandra
qtd_arquivos = !find cassandra_cf -type f | wc -l

# Quantidade de commits do repositorio cassandra
qtd_commits = !cd cassandra_cf && git log --oneline | grep -c .

# Tamanho (MB) da pasta cassandra
tamanho_pasta_mb = !du -hs --block-size=1M cassandra_cf | cut -f1

# Quantidade de colaboradores (devs que commitaram pelo menos uma vez no projeto cassandra)
qtd_colaboradores_commits = !cd cassandra_cf && git log --pretty=format:"%ae" | sort -u | grep -v "noreply.github.com" | wc -l

# Data do primeiro commit do projeto cassandra
data_primeiro_commit =!cd cassandra_cf && git log --reverse --pretty=format:%ad --date=short | head -1

# Data do ultimo commit do projeto cassandra
data_ultimo_commit = !cd cassandra_cf && git log -1 --pretty=format:%ad --date=short

dict_info_repo_cf = {
  'nome':'cassandra_cf',
	'qtd_arquivos':qtd_arquivos,
	'qtd_commits':qtd_commits,
	'tamanho_pasta_mb':tamanho_pasta_mb,
	'qtd_colaboradores_commits':qtd_colaboradores_commits,
	'data_primeiro_commit':data_primeiro_commit,
	'data_ultimo_commit':data_ultimo_commit,
}

df_info_repo_cf = pd.DataFrame(dict_info_repo_cf)
df_info_repo_cf

,nome,qtd_arquivos,qtd_commits,tamanho_pasta_mb,qtd_colaboradores_commits,data_primeiro_commit,data_ultimo_commit
0,cassandra_cf,5186,28431,486,578,2009-03-02,2023-03-10


In [41]:
# Detalha os tipos de arquivos que aparecem no repositório do cassandra
relacao_linguagens = !cd cassandra_cf && cloc .
#relacao_linguagens[checa_indice_language(relacao_linguagens):]
linguagens_encontradas = relacao_linguagens[(checa_indice_language(relacao_linguagens)+2):]

lista_linguagens, lista_files, lista_blank, lista_comment, lista_code = list(), list(), list(), list(), list()

l_temp = []
vl_temp = []
for linha in linguagens_encontradas:
  elementos = linha.split(' ')
  novos_elementos = []
  linguagem_encontrada = []
  valores_da_linguagem = []
  for item in elementos:
    if item != '':
      novos_elementos.append(item)
  linguagem_encontrada.append(novos_elementos[:-4])
  valores_da_linguagem.append(novos_elementos[-4:])
  #print(linguagem_encontrada, valores_da_linguagem)
  l_temp = l_temp + linguagem_encontrada
  vl_temp = vl_temp + valores_da_linguagem

l_temp = l_temp[:-3]
for i, each in enumerate(l_temp):
  print(i,each)

vl_temp = vl_temp[:-3]
for i,each in enumerate(vl_temp):
  print(i,each)
  lista_files.append(each[0])
  lista_blank.append(each[1]) 
  lista_comment.append(each[2]) 
  lista_code.append(each[3])

0 ['Java']
1 ['YAML']
2 ['AsciiDoc']
3 ['Python']
4 ['XML']
5 ['Markdown']
6 ['SVG']
7 ['Bourne', 'Shell']
8 ['ANTLR', 'Grammar']
9 ['Ant']
10 ['Bourne', 'Again', 'Shell']
11 ['Maven']
12 ['make']
13 ['CSS']
14 ['DOS', 'Batch']
15 ['JSON']
16 ['HTML']
17 ['diff']
18 ['reStructuredText']
0 ['3877', '116507', '146654', '613993']
1 ['45', '767', '2563', '330789']
2 ['109', '5592', '11', '20115']
3 ['31', '2460', '3163', '8332']
4 ['33', '201', '478', '3740']
5 ['13', '773', '0', '3387']
6 ['17', '197', '1399', '2937']
7 ['82', '369', '987', '1970']
8 ['3', '267', '288', '1897']
9 ['3', '185', '233', '1787']
10 ['2', '23', '71', '164']
11 ['1', '25', '20', '108']
12 ['2', '20', '20', '70']
13 ['1', '14', '20', '69']
14 ['2', '1', '0', '56']
15 ['6', '0', '0', '52']
16 ['1', '104', '519', '50']
17 ['3', '16', '44', '40']
18 ['2', '22', '31', '40']


In [42]:
dict_dados_linguagens_cf = {
    'linguagens':l_temp, 
    'files':lista_files, 
    'blank':lista_blank,
    'comment':lista_comment,
    'code':lista_code
}

df_dados_linguagens_cf = pd.DataFrame(dict_dados_linguagens_cf)

df_dados_linguagens_cf['files'] = df_dados_linguagens_cf['files'].apply(lambda x: int(x))
df_dados_linguagens_cf['blank'] = df_dados_linguagens_cf['blank'].apply(lambda x: int(x))	
df_dados_linguagens_cf['comment'] = df_dados_linguagens_cf['comment'].apply(lambda x: int(x))	
df_dados_linguagens_cf['code'] = df_dados_linguagens_cf['code'].apply(lambda x: int(x))

df_dados_linguagens_cf

,linguagens,files,blank,comment,code
0,[Java],3877,116507,146654,613993
1,[YAML],45,767,2563,330789
2,[AsciiDoc],109,5592,11,20115
3,[Python],31,2460,3163,8332
4,[XML],33,201,478,3740
5,[Markdown],13,773,0,3387
6,[SVG],17,197,1399,2937
7,"[Bourne, Shell]",82,369,987,1970
8,"[ANTLR, Grammar]",3,267,288,1897
9,[Ant],3,185,233,1787


In [43]:
l_cf = df_info_repo_cf.columns.tolist()
c_c1 = df_info_repo.iloc[0].to_list()
c_cf = df_info_repo_cf.iloc[0].to_list()

dict_repo= {
    'descricao':l_cf,
    'dados_c1':c_c1,
    'dados_cf':c_cf
}

df_repo = pd.DataFrame(dict_repo)
df_repo

,descricao,dados_c1,dados_cf
0,nome,cassandra_c1,cassandra_cf
1,qtd_arquivos,369,5186
2,qtd_commits,1,28431
3,tamanho_pasta_mb,407,486
4,qtd_colaboradores_commits,1,578
5,data_primeiro_commit,2009-03-02,2009-03-02
6,data_ultimo_commit,2009-03-02,2023-03-10
